<a href="https://colab.research.google.com/github/megkotch09/BluebonnetMay2023/blob/main/Single_Women.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas
!pip install matplotlib
!pip install folium
!pip install requests

import pandas as pd
import geopandas as gpd
import folium
import requests
import os
import numbers
import math

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set your Census Bureau API key
api_key = "8a0b76a97aa387f1d801ca752ec8983574f5c307"

# Set the base URL for the API
base_url = "https://api.census.gov/data"

# Set the variables you want to retrieve
variables = ["B12002_095E", "B12002_126E", "B12002_157E", "B12002_172E"]

# Set the geographic parameters
state_fips = "37"  # FIPS code

# Construct the API request URL with parameters
api_url = base_url + "/2021/acs/acs5?get=" + ",".join(variables) + "&for=county%20subdivision:*&in=state:" + state_fips + "&key=" + api_key

# Send the request and get the response
response = requests.get(api_url)

# Parse the response data (assuming it's in JSON format)
data = response.json()

#print(data)

In [ ]:
# Extract the variable names from the first row of the response
headers = data[0]

# Extract the data values from the second row of the response
rows = data[1:]

# Map the variable names to their corresponding values
df = pd.DataFrame(rows, columns=headers)

#display(df) # yas slay queen

df['ALL'] = df['state'] + df['county'] + df['county subdivision']

df = df.astype(int)

df['Total'] = df['B12002_095E'] + df['B12002_126E'] + df['B12002_157E'] + df['B12002_172E']

#df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]], axis=1, inplace=True)
#print(df)

In [ ]:
# Load shapefile data (left sidebar!!)
shapefile_path = "/content/drive/MyDrive/GT/bluebonnet!!/mine - april 2023/North Carolina/cb_2022_37_cousub_500k/cb_2022_37_cousub_500k.shp"

shapefile = gpd.read_file(shapefile_path)

shapefile.drop(shapefile.columns[[3, 4, 6, 7, 8, 9, 10]], axis=1, inplace=True)

#print(shapefile)

json = shapefile.to_json()

df = df.astype(int)
shapefile['GEOID'] = shapefile['GEOID'].astype(int)

In [ ]:
merged_data = shapefile.merge(df, left_on="GEOID", right_on="ALL", how="left")
#print(merged_data)

#df['ALL'] corresponds to shapefile['GEOID']

In [ ]:
# Create a base map centered
map_center = [35.6301, -79.8064]  # Latitude and longitude
m = folium.Map(location=map_center, zoom_start=7)
bins = generate_bins(15)

In [ ]:
# Overlay the demographic data as a choropleth map
folium.Choropleth(
    geo_data=json, #this is supposed to be a json file
    name="choropleth",
    data=merged_data, #this is supposed to be a dataframe
    columns=["ALL", "Total"],
    key_on="feature.properties.GEOID",
    fill_color="Spectral",
    fill_opacity=0.9,
    line_opacity=0.2,
    legend_name="Population",
    bins=bins
).add_to(m)

folium.LayerControl().add_to(m)

# Display the map
#m.save("new_hampshire_demographics_map.html")

m

In [ ]:
def generate_bins(nbins):
  quantiles = []
  for i in range(0, nbins+1):
    quantiles.append(i/nbins)

  bins = list(df['Total'].quantile(quantiles))

  return bins